'''Exports a trained model into a frozen graph'''

/!\ Can't have 2 notebooks with tf.Sessions in parrallel !

Run from environment with requirements:
* humanfriendly

# Freeze graph (.pbtxt to .pb)

In [2]:
import os
import tqdm
import numpy as np

#import tensorflow as tf
from tensorflow.python.tools import freeze_graph

In [3]:
path_to_root = '../../../..'
path_to_outputs = os.path.join(path_to_root,'outputs/fasterRCNN_07_02_img_resize')

pbtxt_path = os.path.join(path_to_outputs, 'graph.pbtxt')
ckpt_path = os.path.join(path_to_outputs, 'model.ckpt-300000')
frozen_graph_path = os.path.join(path_to_outputs, "frozen_graph_MergeSummary.pb")

In [9]:
freeze_graph.freeze_graph(input_graph=pbtxt_path,
                          input_saver='',input_binary=False,
                          input_checkpoint=ckpt_path,
                          output_node_names='Merge/MergeSummary', 
                          #output_node_names = 'group_deps', 
                          #output_node_names ? Found the last node name in pbtxt_file: 'Merge/MergeSummary'
                          # Or a random one in TensorBoard: 'group_deps', 
                          
                          restore_op_name='save/restore_all',
                          filename_tensor_name='save/Const:0',
                          output_graph=frozen_graph_path,
                          clear_devices=True,initializer_nodes='')


INFO:tensorflow:Restoring parameters from ../../../../outputs/fasterRCNN_07_02_img_resize/model.ckpt-300000
INFO:tensorflow:Froze 276 variables.
INFO:tensorflow:Converted 276 variables to const ops.


# Inference

In [4]:
import argparse
import glob
import os
import sys
import time

import PIL
#import humanfriendly
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.image as mpimg
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [12]:
def load_model(checkpoint):
    """
    Load a detection model (i.e., create a graph) from a .pb file
    """

    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(checkpoint, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    
    return detection_graph

load_model(frozen_graph_path)

In [5]:
from run_tf_detector import *

In [26]:
sample_img_path = os.path.join(path_to_root,'data/Sample_imagery_6-7/116592_clipped.png')
sample_img_path

'../../../../data/Sample_imagery_6-7/116592_clipped.png'

In [25]:
frozen_graph_path

'../../../../outputs/fasterRCNN_07_02_img_resize/frozen_graph_MergeSummary.pb'

In [6]:
! python run_tf_detector.py '../../../../outputs/fasterRCNN_07_02_img_resize/frozen_graph_MergeSummary.pb' --imageFile '../../../../data/Sample_imagery_6-7/116592_clipped.png'

Running detector on 1 images
Loading model...
Traceback (most recent call last):
  File "run_tf_detector.py", line 544, in <module>
    main()
  File "run_tf_detector.py", line 539, in main
    confidenceThreshold=args.threshold, outputDir=args.outputDir)
  File "run_tf_detector.py", line 379, in load_and_run_detector
    print("Loaded model in {}".format(humanfriendly.format_timespan(elapsed)))
NameError: name 'humanfriendly' is not defined


Output:
    
    Running detector on 1 images
Loading model...
Loaded model in 1.17 second
Loading images...
100%|################################################################################################################################################################################| 1/1 [00:00<00:00, 14.05it/s]
Finished loading 1 file(s) in 0.07 seconds
Running detector...
0it [00:00, ?it/s]
Traceback (most recent call last):
  File "run_tf_detector.py", line 544, in <module>
    main()
  File "run_tf_detector.py", line 539, in main
    confidenceThreshold=args.threshold, outputDir=args.outputDir)
  File "run_tf_detector.py", line 381, in load_and_run_detector
    boxes,scores,classes,images = generate_detections(detection_graph,imageFileNames)
  File "run_tf_detector.py", line 157, in generate_detections
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/ops.py", line 3666, in get_tensor_by_name
    return self.as_graph_element(name, allow_tensor=True, allow_operation=False)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/ops.py", line 3490, in as_graph_element
    return self._as_graph_element_locked(obj, allow_tensor, allow_operation)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/ops.py", line 3532, in _as_graph_element_locked
    "graph." % (repr(name), repr(op_name)))
KeyError: "The name 'image_tensor:0' refers to a Tensor which does not exist. The operation, 'image_tensor', does not exist in the graph."
root@ac95df47b5f6:/charlie/repos/dams/tfod/detection# cd ../../..


# Aternatives